<a href="https://colab.research.google.com/github/GiovanaThais/listasPython/blob/master/resumo_Compilado_Processamento_De_Imagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando as bibliotecas e pacotes necessários
import random
import scipy.misc
import numpy as np
import cv2 as cv
from PIL import Image
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from skimage import io
from skimage import data
from skimage.transform import resize, rotate
from skimage.color import rgb2gray, rgb2gray
from skimage.color import convert_colorspace
from skimage.filters import threshold_otsu
from scipy import ndimage

In [ ]:
# Função que gera histograma
def get_histogram(imagem, limite_cinza):
  array = imagem.flatten()
  hist = np.zeros(limite_cinza, dtype='int')
  for i in range(0, len(array)):
    hist[array[i]] += 1
  return hist

In [ ]:
# Função que retorna imagem negativa
def imagemNegativa(imagem):
  img = -1 * imagem + 255
  return img

In [ ]:
# Função da transformação Linear
def transformacao_linear(imagem, g_min, g_max):
  f_min = min(imagem.flatten())
  f_max = max(imagem.flatten())

  a = (g_max - g_min) / (f_max - f_min)
  b = g_min

  imagem = imagem - f_min

  novaImagem = (a * imagem) + b

  return np.uint8(novaImagem)

In [ ]:
# Função que aplica correlação na imagem
def correlacao(img, img_corr, mascara):
  img_correlacao = np.zeros(np.array(img.shape) + np.array(mascara.shape) - 1)

  for x in range(mascara.shape[0]):
    for y in range(mascara.shape[1]):
      img_correlacao[x:x+img.shape[0], y:y+img.shape[1]] += img * mascara[x,y]

      return img_correlacao.astype('uint8')

In [ ]:
# Função que aplica a convolução na imagem e funções agregadas

def rotate_180(image):

  size_image = image.shape
  M = size_image[0]
  N = size_image[1]

  new_image = np.zeros((M,N))

  for i in range (0, M):
    for j in range(0, N):
      new_image[(M-1)-i, (N-1)-j] = image[i,j]

  return new_image

def get_slice(point, size, image):

  delta = int(size/2)

  a = (point[0] - delta)
  b = (point[1] - delta)

  return image[a:(a+size), b:(b+size)]

def convolucao(imagem, mascara):

  size_imagem = imagem.shape
  M = size_imagem[0]
  N = size_imagem[1]

  size_mascara = mascara.shape
  m = size_mascara[0]
  n = size_mascara[1]

  nova_imagem = np.zeros(imagem.shape)

  delta = int(m/2)

  for i in range(0+delta, M-delta):
    for j in range(0+delta, N-delta):
      slice_imagem = get_slice((i,j), m, imagem)
      mascara_180 = rotate_180(mascara)
      new_i_j = np.sum(slice_imagem * mascara_180)
      nova_imagem[i,j] = new_i_j

  return nova_imagem

In [ ]:
#Carrega e exibe a imagem
img01 = cv.imread('/content/img01.jpg')

cv2_imshow(img01)

In [ ]:
#Carrega e exibe a imagem
img02 = cv.imread('img02.jpg')

cv2_imshow(img02)

In [ ]:
# Gera o histograma com a função calcHist do openCV
h = cv.calcHist([img01], [0], None, [256], [0,256])

plt.figure()
plt.title("Histograma Imagem Cinza")
plt.xlabel("Intensidade")
plt.ylabel("Quantidade de pixels")

plt.plot(h)
plt.xlim([0,256])
plt.show()

In [ ]:
# Exibe a Altura, largura e canais da imagem
print('Altura  :',img01.shape[0])
print('Largura :',img01.shape[1])
print('Canais  :',img01.shape[2])

Altura  : 961
Largura : 741
Canais  : 3


In [ ]:
# Maximo, Minimo, Média de pixels
print('Maximo :',img01.max())
print('Minimo :',img01.min())
print('Média  :',img01.mean())

Maximo : 255
Minimo : 0
Média  : 47.85779124028755


In [ ]:
# Redimensiona o tamanho da imagem
img01_re = resize(img01, (128, 128))
plt.imshow(img01_re)
plt.show() 

In [ ]:
# Rotaciona a imagem em um certo angulo
img01_rot = rotate(img01, 90, resize=True)
plt.imshow(img01_rot)
plt.show() 

In [ ]:
# Converte a imagem de RGB para HSV
img02_cinza = cv.cvtColor(img02, cv.COLOR_BGR2GRAY)
cv2_imshow(img02_cinza) 

In [ ]:
# Gera o histograma da imagem
h = cv.calcHist([img02_cinza], [0], None, [256], [0,256])

plt.figure()
plt.title("Histograma Imagem Cinza")
plt.xlabel("Intensidade")
plt.ylabel("Quantidade de pixels")

plt.plot(h)
plt.xlim([0,256])
plt.show()

In [ ]:
# Converte a imagem de RGB para HSV
img02_hsv = convert_colorspace(img02, 'RGB', 'HSV')
plt.imshow(img02_hsv)
plt.show() 

In [ ]:
# Calculo de vizinhança 4
def vizinhanca4(linha, coluna):
    v1 = (linha - 1, coluna)
    v2 = (linha, coluna - 1)
    v3 = (linha + 1, coluna)
    v4 = (linha, coluna + 1)

    return [v1, v2, v3, v4]

In [ ]:
# Calculo de vizinhança 8
def vizinhanca8(linha, coluna):
    v5 = (linha - 1, coluna - 1)
    v6 = (linha - 1, coluna + 1)
    v7 = (linha + 1, coluna - 1)
    v8 = (linha + 1, coluna + 1)
    
    vizinhanca = vizinhanca4(linha, coluna) + [v5, v6, v7, v8]
    return vizinhanca

In [ ]:
# Encontrar os vizinhos dos pixels (5,5), (50,50), (100,100), (200,200), (250,250)

v1 = vizinhanca8(5, 5)
v2 = vizinhanca8(50, 50)
v3 = vizinhanca8(100, 100)
v4 = vizinhanca8(200, 200)
v5 = vizinhanca8(250, 250)

v_list = [v1,v2,v3,v4,v5]

for i in v_list:
  for j in i:
    print(f"Imagem 1 - pixel {j}: {img01[j[0]][j[1]]}")

Imagem 1 - pixel (4, 5): [6 6 6]
Imagem 1 - pixel (5, 4): [7 7 7]
Imagem 1 - pixel (6, 5): [6 6 6]
Imagem 1 - pixel (5, 6): [6 6 6]
Imagem 1 - pixel (4, 4): [7 7 7]
Imagem 1 - pixel (4, 6): [6 6 6]
Imagem 1 - pixel (6, 4): [7 7 7]
Imagem 1 - pixel (6, 6): [6 6 6]
Imagem 1 - pixel (49, 50): [7 7 7]
Imagem 1 - pixel (50, 49): [6 6 6]
Imagem 1 - pixel (51, 50): [9 9 9]
Imagem 1 - pixel (50, 51): [7 7 7]
Imagem 1 - pixel (49, 49): [9 9 9]
Imagem 1 - pixel (49, 51): [8 8 8]
Imagem 1 - pixel (51, 49): [11 11 11]
Imagem 1 - pixel (51, 51): [9 9 9]
Imagem 1 - pixel (99, 100): [16 16 16]
Imagem 1 - pixel (100, 99): [15 15 15]
Imagem 1 - pixel (101, 100): [15 15 15]
Imagem 1 - pixel (100, 101): [20 20 20]
Imagem 1 - pixel (99, 99): [16 16 16]
Imagem 1 - pixel (99, 101): [16 16 16]
Imagem 1 - pixel (101, 99): [11 11 11]
Imagem 1 - pixel (101, 101): [21 21 21]
Imagem 1 - pixel (199, 200): [24 24 24]
Imagem 1 - pixel (200, 199): [18 18 18]
Imagem 1 - pixel (201, 200): [10 10 10]
Imagem 1 - pixel (2

In [ ]:
img01_ruido = np.asarray(cv.imread('img01.jpg'))
tamanho_img01 = img01_ruido.shape

M = tamanho_img01[0]
N = tamanho_img01[1]

new_image = np.zeros((M, N), dtype='uint8')

for i in range(0, M):
  randomValues = random.sample(range(0, N), 40)
  for value in randomValues:
    img01_ruido[i, value] = 0
plt.imshow(img01_ruido, cmap="gray")
plt.show()


In [ ]:
# Remove ruidos da imagem
median_img01 = cv.medianBlur(img01, 5)
plt.subplot(121), plt.imshow(img01_ruido), plt.title('Com ruído')
plt.xticks([]), plt.yticks([])

plt.subplot(122), plt.imshow(median_img01), plt.title('Sem ruído')
plt.xticks([]), plt.yticks([])

plt.show() 

In [ ]:
# Chama a função get_histogram()
histograma_img01 = get_histogram(img01, 256)
x_img01 = range(len(histograma_img01))
y_img01 = histograma_img01

histograma_img02 = get_histogram(img02, 256)
x_img02 = range(len(histograma_img02))
y_img02 = histograma_img02

# Exibe o histograma em gráfico
plt.clf()
plt.figure(figsize=(15,5))

plt.subplot(121)
plt.plot(x_img01, y_img01)

plt.subplot(122)
plt.plot(x_img02, y_img02)


In [ ]:
# Exibe a matriz do histograma
print(histograma_contraste)

In [ ]:
# Utiliza a função que retorna a imagem negativa
img01_neg = imagemNegativa(img01)
cv2_imshow(img01_neg)

In [ ]:
# Carrega a imagem na escala cinza
img02_cinza = cv.cvtColor(img02, cv.COLOR_BGR2GRAY)
cv2_imshow(img02_cinza) 

In [ ]:
# Aplica a função de transformação linear na imagem
img02_linear = transformacao_linear(img02_cinza,0,255)

cv2_imshow(img02_linear)

In [ ]:
# Chama a função get_histogram()
histograma_img02_cinza = get_histogram(img02_cinza, 256)
x_img02_cinza = range(len(histograma_img02_cinza))
y_img02_cinza = histograma_img02_cinza

histograma_img02_linear = get_histogram(img02_linear, 256)
x_img01_linear = range(len(histograma_img02_linear))
y_img01_linear = histograma_img02_linear

# Exibe o histograma em gráfico
plt.clf()
plt.figure(figsize=(15,5))

plt.subplot(121)
plt.plot(x_img01, y_img01)

plt.subplot(122)
plt.plot(x_img01_linear, y_img01_linear)

In [ ]:
# Gera o histograma equalizado pela função da biblioteca cv
hist_eq_img02 = cv.equalizeHist(img02_cinza)

plt.figure()
plt.title('Histograma original equalizado')
plt.xlabel('Intensidade')
plt.ylabel('Quantidade de pixels')
plt.hist(hist_eq_img02.ravel(), 256, [0,256])
plt.xlim([0,256])
plt.show()

In [ ]:
# Realiza a transformação por raiz quadrada
img01_raiz = cv.imread('img01.jpg')

# transforma a matriz em float para permitir valores negativos
img_float_raiz = np.float32(img01_raiz)

# aplica a transformação pela raiz quadrada
img01_raiz = np.sqrt(img_float_raiz)

# normaliza a matriz para o intervalo 0-255
img_norm_raiz = cv.normalize(img01_raiz, None, 0, 255, cv.NORM_MINMAX)

# converte de volta para um tipo inteiro de 8 bits
img_final_raiz = np.uint8(img_norm_raiz)

# exibe o resultado final
cv2_imshow(img_final_raiz)


In [ ]:
# Transformação por mapeamento quadrado
img01_quad = cv.imread('img01.jpg')


# transforma a matriz em float para permitir valores negativos
img_float_quad = np.float32(img01_quad)

# aplica a transformação pelo mapeamento quadrado
img01_quad = img_float_quad ** 2

# normaliza a matriz para o intervalo 0-255
img_norm_quad = cv.normalize(img01_quad, None, 0, 255, cv.NORM_MINMAX)

# converte de volta para um tipo inteiro de 8 bits
img_final_quad = np.uint8(img_norm_quad)

# exibe o resultado final
cv2_imshow(img_final_quad)

In [ ]:
# Gera o histograma da imagem por mapeamento e raiz, depois os compara
histograma_img01_raiz = get_histogram(img_final_raiz, 256)
x_img01_raiz = range(len(histograma_img01_raiz))
y_img01_raiz = histograma_img01_raiz

histograma_img01_quad = get_histogram(img_final_quad, 256)
x_img01_quad = range(len(histograma_img01_quad))
y_img01_quad = histograma_img01_quad

# Exibe o histograma em gráfico
plt.clf()
plt.figure(figsize=(15,5))

plt.subplot(121)
plt.plot(x_img01_raiz, y_img01_raiz)

plt.subplot(122)
plt.plot(x_img01_quad, y_img01_quad)

In [ ]:
# Aplica a função de correlação na imagem
img01_corr = cv.imread('img01.jpg',0)
img01_original = cv.imread('img01.jpg',0)

mascara_corr = np.array([ [1,1,1], [1,1,1], [1,1,1] ])

img01_correlacao = correlacao(img01_original,img01_corr,mascara_corr)

cv2_imshow(img01_correlacao)


In [ ]:
# Aplica a função de convolução na imagem
mascara_conv = np.array([ [-1,0,1], [-2,1,2], [-1,0,1] ])

img01_convolucao = convolucao(img01_original, mascara_conv)
cv2_imshow(img01_convolucao)

In [ ]:
#Aplica o filtro de passa baixa : média

h1 = 1/9 * np.ones((3,3), dtype='int')
h2 = 1/25 * np.ones((5,5), dtype='int')
h3 = 1/225 * np.ones((15,15), dtype='int')

figure_size = 9
img01_media_filtered = cv.blur(img01, (figure_size, figure_size))

plt.figure(figsize=(11,6))
plt.subplot(121)
plt.imshow(img01)
plt.title('Imagem original')
plt.xticks([])
plt.yticks([])

plt.subplot(122)
plt.imshow(img01_media_filtered)
plt.xticks([])
plt.yticks([])
plt.title('Passa-baixa: media')

plt.show()

In [ ]:
# Aplica o filtro de passa-baixa : média ponderada

img01_mediap_filtered = cv.filter2D(img01, -1, h1)

plt.figure(figsize=(11,6))
plt.subplot(121)
plt.imshow(img01)
plt.title('Imagem original')
plt.xticks([])
plt.yticks([])

plt.subplot(122)
plt.imshow(img01_mediap_filtered)
plt.xticks([])
plt.yticks([])
plt.title('Passa-baixa: media ponderada')


In [ ]:
# Aplica o filtro de passa-baixa : moda

kernel = np.ones((3, 3), np.float32) / 9

# Definir o tamanho da janela do filtro de moda
window_size = 5

# Aplicar o filtro de moda na imagem
img01_moda_filtered = ndimage.generic_filter(img01, np.nanmedian, size=window_size)


plt.figure(figsize=(11,6))
plt.subplot(121)
plt.imshow(img01)
plt.title('Imagem original')
plt.xticks([])
plt.yticks([])

plt.subplot(122)
plt.imshow(img01_moda_filtered)
plt.xticks([])
plt.yticks([])
plt.title('Passa-baixa: moda')

In [ ]:
# Aplica o filtor de passa-alta de Prewitt e de Roberts

# Ler a imagem original
img01_p_alta = cv.imread('img01.jpg', 0)

# Definir as máscaras do filtro de Prewitt
prewitt_x = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
prewitt_y = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])

# Aplicar o filtro de Prewitt na imagem
filtered_img_prewitt_x = cv.filter2D(img01_passa, -1, prewitt_x)
filtered_img_prewitt_y = cv.filter2D(img01_passa, -1, prewitt_y)

# Definir as máscaras do filtro de Roberts
roberts_x = np.array([[0, 0, 0], [0, 1, 0], [0, 0, -1]])
roberts_y = np.array([[0, 0, 0], [0, 0, 1], [0, -1, 0]])

# Aplicar o filtro de Roberts na imagem
filtered_img_roberts_x = cv.filter2D(img01_passa, -1, roberts_x)
filtered_img_roberts_y = cv.filter2D(img01_passa, -1, roberts_y)

# Exibir a imagem original e as imagens filtradas
print('Imagem original')
cv2_imshow(img01_passa)
print('Imagem filtrada - Prewitt horizontal')
cv2_imshow( filtered_img_prewitt_x)
print('Imagem filtrada - Prewitt vertical')
cv2_imshow( filtered_img_prewitt_y)
print('Imagem filtrada - Roberts diagonal 1')
cv2_imshow( filtered_img_roberts_x)
print('Imagem filtrada - Roberts diagonal 2')
cv2_imshow(filtered_img_roberts_y)


In [ ]:
# Aplica o filtro de passa-alta de sobel

sobelx = cv.Sobel(img01_p_alta, cv.CV_64F,1,0)
sobely = cv.Sobel(img01_p_alta, cv.CV_64F,0,1)
sobelxy = cv.Sobel(img01_p_alta, cv.CV_64F,1,1)

print('Imagem original')
cv2_imshow(img01_p_alta)

print()

print('Passa-alta: sobel')
cv2_imshow(sobelxy)